<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-библиотек-и-данных" data-toc-modified-id="Загрузка-библиотек-и-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка библиотек и данных</a></span></li><li><span><a href="#Ознакомление-с-данными" data-toc-modified-id="Ознакомление-с-данными-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Ознакомление с данными</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Выбор-способов-предобработки-данных" data-toc-modified-id="Выбор-способов-предобработки-данных-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Выбор способов предобработки данных</a></span></li><li><span><a href="#Фильтрация-символов-и-лемматизация-токенов" data-toc-modified-id="Фильтрация-символов-и-лемматизация-токенов-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Фильтрация символов и лемматизация токенов</a></span></li><li><span><a href="#Векторизация-текстов" data-toc-modified-id="Векторизация-текстов-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Векторизация текстов</a></span></li></ul></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Градиентный-бустинг-(CatBoost)" data-toc-modified-id="Градиентный-бустинг-(CatBoost)-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Градиентный бустинг (CatBoost)</a></span></li><li><span><a href="#Анализ-лучших-моделей" data-toc-modified-id="Анализ-лучших-моделей-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Анализ лучших моделей</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Загрузка библиотек и данных

In [1]:
# Загразука библиотек
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)
import matplotlib.pyplot as plt

import re
from nltk.stem.wordnet import WordNetLemmatizer as wordnet_lemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

In [2]:
# Загрузка данных

try:
    comments_data = pd.read_csv('/datasets/toxic_comments.csv', 
                                error_bad_lines=False, 
                                index_col = 0)

except:
    comments_data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', 
                                error_bad_lines=False , 
                                index_col = 0)
    
comments_data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159446,""":::::And for the second time of asking, when ...",0
159447,You should be ashamed of yourself \n\nThat is ...,0
159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159449,And it looks like it was actually you who put ...,0


Данные содержат около 100 строк с ошибками форматирования, поэтому они были пропущены (параметр `error_bad_lines=False`). Доля пропущенных строк пренебрежимо мала, поэтому не должна оказать существенного влияния на результат.

### Ознакомление с данными

Проверим отсутствие пропусков и дубликатов:

In [3]:
print("Доля пропусков:")
display(comments_data.isna().mean())

print(f"Доля дубликатов: {comments_data.duplicated().mean()*100:0.2f} %")

Доля пропусков:


text     0.0
toxic    0.0
dtype: float64

Доля дубликатов: 0.00 %


Пропуски и дубликаты отсутствуют.  
Проверим долю токсичных комментариев:

In [4]:
comments_data.toxic.values.mean().round(4)

0.1016

Выборка не сбалансирована относительно числа токсичных комментариев. Это обстоятельство должно быть учтено при построении модели.  
Записи в начале и конце выборки написаны на английском языке. Для выбора адекватного подхода при предобработке рассмотрим распространенность символов в комментариях:

In [5]:
all_chars = dict()
N_COMMENTS = 20_000

for comment in comments_data.sample(N_COMMENTS, replace=False).text.values:
    for char in comment:
        if all_chars.get(char.lower()):
            all_chars[char.lower()] = all_chars.get(char.lower()) + 1 / N_COMMENTS
        else:
            all_chars[char.lower()] = 1 / N_COMMENTS
            
pd.DataFrame({'symbol' : all_chars.keys(),
             'counts per comment': all_chars.values()}).sort_values(by='counts per comment', 
                                                                    ascending=False).head(80)

,symbol,counts per comment
1,,67.86015
5,e,36.65465
7,t,28.68060
2,a,25.32435
0,i,25.08825
...,...,...
112,‎,0.00525
51,$,0.00465
129,<,0.00350
169,á,0.00345


В 50 наиболее популярных символов входят только символы латинского алфавита, цифры, специальные символы и знаки пунктуации. Символы других алфавитов встречаются не чаще 3 раз на 1000 комментариев. Даже если такие символы не случайны, а комментарии действительно написаны не на английском языке, количество таких комментариев недостаточно для построения качественной модели на их основе. Таким образом, при предобработке могут быть исключены все буквенные символы кроме символов латинского алфавита.

### Предобработка данных

#### Выбор способов предобработки данных

Требования к итоговой модели относительно низкие (F1-метрика выше 0,75), поэтому для её создания могут быть использованы классические методы машинного обучения. Предобработка в таком случае будет включать следующие этапы:
- очистка текста от "неправильных" символов;
- лемматизация/стеммизация токенов;
- составление n-грамм;
- векторизация текста (tf-idf).

####  Фильтрация символов и лемматизация токенов
Исключим числа и символы кроме символов латинского алфавита, а также приведем записи к нижнему регистру:

In [6]:
comments_data.text = comments_data.text.apply(lambda x: ' '.join(re.sub("([^\w\']|[\d])", " ", x).split()).lower())
comments_data

,text,toxic
0,explanation why the edits made under my userna...,0
1,d'aww he matches this background colour i'm se...,0
2,hey man i'm really not trying to edit war it's...,0
3,more i can't make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0
...,...,...
159446,and for the second time of asking when your vi...,0
159447,you should be ashamed of yourself that is a ho...,0
159448,spitzer umm theres no actual article for prost...,0
159449,and it looks like it was actually you who put ...,0


Приведем все слова в комментариях к исходной форме:

In [7]:
wn = wordnet_lemmatizer()

comments_data.text = comments_data.text.apply(lambda x: " ".join(wn.lemmatize(word) for word in x.split()))
comments_data.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,d'aww he match this background colour i'm seem...,0
2,hey man i'm really not trying to edit war it's...,0
3,more i can't make any real suggestion on impro...,0
4,you sir are my hero any chance you remember wh...,0


####  Векторизация текстов

Учитывая низкие требования, удовлетворительное качество модели может быть получено за счет учета обсценной лексики и прямых оскорблений. В таком случае не требуется развитый учет контекста, поэтому будет достаточно tf-idf векторизации. Поскольку токсичность комментария зависит от его направленности (на самого говорящего (самокритика) или собеседника) использование стоп-слов в данном случае нежелательно.  

Чтобы условия проверки моделей соответствовали реальности векторизация должна обучаться только на тренировочном наборе данных, поэтому заранее поделим данные на тренировочный, тестовый и валидационный наборы (учитывая большой объем данных можно обойстись проверкой на отложенной выборке, без кроссвалидации):

In [8]:
train_val, test = train_test_split(comments_data, test_size=0.4, random_state = 123)
train, val = train_test_split(train_val, test_size=0.2, random_state = 123)

Максимальную частоту встречаемости токенов при векторизации следует установить близкой по порядку к доле позитивного (токсичного) класса, а минимальную - порядка нескольких единиц или десятков для исключения случайных и слишком редких слов. Отдельно построим представления для униграмм, а также униграмм и биграмм:

In [9]:
# Инициализируем два векторизатора для исключительно униграмм и униграмм и биграмм:
vectorizer_uni = TfidfVectorizer(max_df=0.5, min_df=30, ngram_range=(1, 1))
vectorizer_bi = TfidfVectorizer(max_df=0.5, min_df=30, ngram_range=(1, 2))

for vec in [vectorizer_uni, vectorizer_bi]:
    vec.fit(train.text)

In [10]:
# Преобразованные записи сохраним в списки по три:
comments_unigramm = []
comments_bigramm = []

for part in [train, val, test]:
    comments_unigramm.append(vectorizer_uni.transform(part.text))
    comments_bigramm.append(vectorizer_bi.transform(part.text))

Текстовые данные подготовлены. Можно переходить к обучению моделей.

## Обучение

### Логистическая регрессия

Линейные модели хорошо подходят для работы категориальными признаками (результат зависит от наличия или отсутствия определенной категории), поэтому можно ожидать неплохого результата при бинарной классификации тональности текстов. Кроме того, линейные модели обладают наибольшой скоростью работы. Проверим модель логистической регрессии на текстах кодированных при помощи униграмм:

In [11]:
model_lr_uni = LogisticRegression(max_iter=300, random_state=123)
model_lr_uni.fit(comments_unigramm[0], train.toxic)
f1_score(val.toxic, model_lr_uni.predict(comments_unigramm[1])).round(3)

0.731

Качество модели близко к предъявленным требованиям. Поскольку данные несбалансированы, для максимизации F1-метрики можно воспользоваться подбором порога классификации. Напишем функцию для подбора порога классификации:

In [12]:
def thresh_opti(model, test_t, test_f):
    best_thresh = 0
    score = 0
    pred = model.predict_proba(test_f)
    
    for thresh in np.linspace(0, 1):
        cur_score = f1_score(test_t, pred[:,1]>thresh)
        if cur_score > score:
            score = cur_score
            best_thresh = thresh
    print(f"Лучшее значение F1-метрики: {score:0.3f} при пороге {best_thresh:0.3f}")
    return best_thresh

Определим лучший порог при использовании униграмм на тренировочной выборке:

In [13]:
best_thres_lr_uni = thresh_opti(model_lr_uni, val.toxic, comments_unigramm[1])

Лучшее значение F1-метрики: 0.778 при пороге 0.286


Проверим модель с использованием представления, включающего биграммы:

In [14]:
model_lr_bi = LogisticRegression(max_iter=300, random_state=123)
model_lr_bi.fit(comments_bigramm[0], train.toxic)
f1_score(val.toxic, model_lr_bi.predict(comments_bigramm[1])).round(3)

0.712

Качество модели, обученной на представленнии, влючающем биграммы, хуже, чем у более простой модели. Возможно это связано с трудностью отбора значимых признаков и переобучением. Попробуем улучшить результат подбором порога классификации:

In [15]:
best_thres_lr_bi = thresh_opti(model_lr_bi, val.toxic, comments_bigramm[1])

Лучшее значение F1-метрики: 0.770 при пороге 0.245


Метрика снова оказалась хуже чем для более простой модели. Следовательно предпочтение следует отдать более простой модели.

### Градиентный бустинг (CatBoost)

Рассмотрим более сложную модель - градиентный бустинг. Для снижения риска переобучения и ускорения обучения модели ограничим число итераций, а чтобы учесть дисбаланс классов, установим параметр 'auto_class_weights' равным 'Balanced'.

In [16]:
model_gb = CatBoostClassifier(iterations=500, 
                              verbose=100,
                              eval_metric='F1',
                              early_stopping_rounds=50,
                              auto_class_weights='Balanced',
                              random_seed=123)

model_gb.fit(comments_unigramm[0],
             train.toxic,
             eval_set=(comments_unigramm[1], val.toxic))

f1_score(val.toxic, model_gb.predict(comments_unigramm[1])).round(3)

Learning rate set to 0.125045
0:	learn: 0.6538862	test: 0.6682767	best: 0.6682767 (0)	total: 1.81s	remaining: 15m 5s
100:	learn: 0.8764789	test: 0.8647282	best: 0.8651693 (99)	total: 2m 6s	remaining: 8m 20s
200:	learn: 0.9108431	test: 0.8785633	best: 0.8786710 (197)	total: 4m	remaining: 5m 57s
300:	learn: 0.9291811	test: 0.8818204	best: 0.8827146 (256)	total: 5m 53s	remaining: 3m 53s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8827146475
bestIteration = 256

Shrink model to first 257 iterations.


0.713

Попробуем улучшить целевую метрику подбором порога классификации:

In [17]:
best_thres_gb_uni = thresh_opti(model_gb, val.toxic, comments_unigramm[1])

Лучшее значение F1-метрики: 0.766 при пороге 0.653


Значение метрики хуже, чем для модели логистической регрессии, при значительно большем времени обучения и предсказания.  
`CatBoost` отличается от прочих наличием встроенной поддержки текстовых признаков, для построения биграмм воспользуемся встроенными возможностям:

In [18]:
model_cat = CatBoostClassifier(iterations=500, 
                               verbose=100,
                               text_features=['text'],
                               eval_metric='F1',
                               early_stopping_rounds=50,
                               auto_class_weights='Balanced',
                               dictionaries=[{"dictionary_id" : "BiGram",
                                              "gram_order" : "2"
                                             }, 
                                             {"dictionary_id" : "Word",
                                              "gram_order" : "1"
                                             }],
                               random_seed=123)

model_cat.fit(train.drop('toxic', axis=1), 
              train.toxic, 
              eval_set=(val.drop('toxic', axis=1), val.toxic))

f1_score(val.toxic, model_cat.predict(val.drop('toxic', axis=1))).round(3)

Learning rate set to 0.125045
0:	learn: 0.8369804	test: 0.8475891	best: 0.8475891 (0)	total: 420ms	remaining: 3m 29s
100:	learn: 0.8952252	test: 0.9000996	best: 0.9000996 (100)	total: 40.8s	remaining: 2m 41s
200:	learn: 0.9123847	test: 0.9029208	best: 0.9032351 (198)	total: 1m 20s	remaining: 2m
300:	learn: 0.9236196	test: 0.9047701	best: 0.9053888 (293)	total: 2m	remaining: 1m 19s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9053888248
bestIteration = 293

Shrink model to first 294 iterations.


0.718

Время обучения модели существенно сократилось в сравнении с использовнием внешней векторизации текста. F1-метрика модели оказалась снова хуже полученной для логистической регрессии. Определим порог классификации, максимизирующий целевую метрику:

In [19]:
best_thres_cat_bi = thresh_opti(model_cat, val.toxic, val.drop('toxic', axis=1))

Лучшее значение F1-метрики: 0.789 при пороге 0.796


После подбора порога классификации значение целевой метрики превысило значение метрики для логистической регрессии. Учитывая очень небольшую разницу в F1-метрике, для выбора лучшей модели дополнительно сравним точность моделей:

In [20]:
from sklearn.metrics import accuracy_score
accuracy_lr = accuracy_score(val.toxic, 
                             model_lr_uni.predict_proba(comments_unigramm[1])[:,1]>best_thres_lr_uni)

accuracy_cat_bi = accuracy_score(val.toxic, 
                             model_cat.predict_proba(val.drop('toxic', axis=1))[:,1]>best_thres_cat_bi)

print("Точность при оптимальном для F1-метрики пороге:")
print(10*'.', f"{accuracy_lr:0.4f} - логистическая регрессия")
print(10*'.', f"{accuracy_cat_bi:0.4f} - градиентный бустинг (CatBoost)")

Точность при оптимальном для F1-метрики пороге:
.......... 0.9567 - логистическая регрессия
.......... 0.9575 - градиентный бустинг (CatBoost)


Точность модели градиентного бустинга превышает точность логистической регрессии менее чем на 0.001. 
Учитывая разную сложность моделей, разное время их обучения и разное время предсказания, в данном случае предпочтение следует отдать более простой модели - логистической регрессии.

### Анализ лучших моделей

Как было написано выше, в данном случае, при использовании tf-idf векторизации лучшей моделью является модель логистической регрессии. Объяснением хорошей работы линейной модели в данном проекте является то, что модель фактически работает с качественными признаками - наличием или отсутствием определенных "токсичных" слов в комментарии. 
Качество предсказаний градиентного бустинга лишь немного превышает качество логистической регрессии, при значительно более высокой сложности модели.  
Можно предпложить что модель относит к токсичным комментарии, содержащий нецензурную, грубую и т.д. лексику. Проверим, какие признаки являются наиболее значимыми для лучшей модели:

In [21]:
coeffs = model_lr_uni.coef_[0]

In [22]:
vocab = vectorizer_uni.vocabulary_
vocab = pd.DataFrame({'name':vocab.keys()}, index=vocab.values()
                    ).sort_index()
vocab['coeff']=coeffs
print('Слова характеризующие токсичность комментария:', '\n')
print(vocab.sort_values(by='coeff').tail(50)['name'].values)

print('\n', 'Слова характеризующие "антитоксичность" комментария:', '\n')
print(vocab.sort_values(by='coeff').head(50)['name'].values)

Слова характеризующие токсичность комментария: 

['scum' 'retarded' 'homosexual' 'sick' 'kill' 'nazi' 'fat' 'fuckin' 'hate'
 'ignorant' 'wtf' 'pussy' 'piss' 'retard' 'fool' 'fucker' 'pathetic'
 'racist' 'fag' 'cock' 'sex' 'dumbass' 'dumb' 'you' 'fucked' 'die' 'loser'
 'shut' 'jerk' 'damn' 'gay' 'penis' 'nigger' 'bastard' 'moron' 'hell'
 'faggot' 'cunt' 'crap' 'bullshit' 'dick' 'asshole' 'bitch' 'suck' 'as'
 'stupid' 'idiot' 'shit' 'fucking' 'fuck']

 Слова характеризующие "антитоксичность" комментария: 

['thank' 'thanks' 'talk' 'please' 'if' 'best' 'utc' 'welcome' 'for'
 'redirect' 'help' 'appreciate' 'section' 'cheer' 'consensus' 'may'
 'article' 'see' 'could' 'point' 'continue' 'issue' 'agree' 'support'
 'request' 'interested' 'under' 'version' 'there' 'from' 'tag' 'at'
 'reference' 'sure' 'in' 'but' 'provide' 'list' 'need' 'ask' 'apologize'
 'number' 'dispute' 'source' 'case' 'wp' 'review' 'didn' 'involved'
 'started']


Это действительно так. Отсюда сразу следует уязвимость построенных моделей: добавление позитивных слов может скрыть от модели явно негативный комментарий, например:

In [23]:
test_string = ['thank you stupid jerk please welcome in this section for help and interested reference']
test_string_vec = vectorizer_uni.transform(test_string)
print('Фраза:', test_string[0], '\n')
print("Предсказание логистической регрессии:", model_lr_uni.predict(test_string_vec)[0])
print("Предсказание градиентного бустинга:", model_cat.predict(test_string))

Фраза: thank you stupid jerk please welcome in this section for help and interested reference 

Предсказание логистической регрессии: 0
Предсказание градиентного бустинга: 0


Обе модели были обмануты, сократим количество позитивных слов:

In [24]:
test_string = ['thank you stupid jerk please welcome in this section for help and reference']
test_string_vec = vectorizer_uni.transform(test_string)
print('Фраза:', test_string[0], '\n')
print("Предсказание логистической регрессии:", model_lr_uni.predict(test_string_vec)[0])
print("Предсказание градиентного бустинга:", model_cat.predict(test_string))

test_string = ['thank you stupid jerk please welcome in this section for help']
test_string_vec = vectorizer_uni.transform(test_string)

print('\nФраза:', test_string[0], '\n')
print("Предсказание логистической регрессии:", model_lr_uni.predict(test_string_vec)[0])
print("Предсказание градиентного бустинга:", model_cat.predict(test_string))

Фраза: thank you stupid jerk please welcome in this section for help and reference 

Предсказание логистической регрессии: 0
Предсказание градиентного бустинга: 1

Фраза: thank you stupid jerk please welcome in this section for help 

Предсказание логистической регрессии: 1
Предсказание градиентного бустинга: 1


В большинстве случаев логистическая регрессия и градиентный бустинг работают одинаково (одинаково хорошо и одинаково плохо).  
Немного исправить ситуацию можно обучая модели на n-граммах большего чем 1 порядка, ограничив при этом словарь только самыми значимыми позитивными и негативными словами. Выделим 500 слов с наибольшим абсолютным значением коэффициента в линейной модели и заново векторизуем тексты, используя только эти слова в словаре:

In [25]:
vocab['acoeff'] = vocab.coeff.abs()
top_words_500 = vocab.sort_values(by='acoeff').tail(500)['name'].values

vectorizer_top = TfidfVectorizer(ngram_range=(1, 2), vocabulary=top_words_500)

comments_top_words = []
vectorizer_top.fit(train.text)
for part in [train, val, test]:
    comments_top_words.append(vectorizer_top.transform(part.text))

Заново обучим модель логистической регрессии:

In [26]:
model_lr_top = LogisticRegression(random_state=123)
model_lr_top.fit(comments_top_words[0], train.toxic)
f1_score(val.toxic, model_lr_top.predict(comments_top_words[1])).round(3)

0.764

In [28]:
best_thres_lr_top = thresh_opti(model_lr_top, val.toxic, comments_top_words[1])

Лучшее значение F1-метрики: 0.780 при пороге 0.347


Удалось дополнительно улучшить целевую метрику. Требуемый порог целевой метрики удалось преодолеть используя только 500 слов, однако улучшить ситуацию с ложно-отрицательными предсказаниями это не помогает.  Проверим итоговую модель на отложенной тестовой выборке:


In [29]:
f1_score(test.toxic, 
         model_lr_top.predict_proba(comments_top_words[2])[:, 1] > best_thres_lr_top).round(3)

0.784

Значение F1-метрики на тестовой выборке превосходит 0.75, таким образом, выбранная модель удовлетворяет предъявленным к ней требованиям.

## Выводы

1. Требуемый порог F1-метрики может быть достигнут при использовании модели логистической регрессии и tf-idf векторизации текста комментариев.
2. Использование градиентного бустинга слабо улучшает целевую метрику при использовании tf-idf векторизации.
3. Повысить точность предсказаний и ускорить обучение и работу моделей можно ограничив словарь только обсценной, грубой и т.д. лексикой.
4. Общей проблемой исследованных моделей при tf-idf векторизации текста является легкость получения ложноотрицательных предсказаний. Добавление "позитивных", в том числе бесмысленных, слов маскирует явно токсичные комментарии.
5. При необходимости повысить качество моделей можно путем улучшения векторизации - качественный отбор словаря, учет контекста, создание представлений с помощью нейросетей (BERT и т.д.)